# Airworthiness Directives (ADs) - Febriyeni Susi

**Case:**
Build an automated pipeline that
1. Extracts applicability rules from AD PDFs (not manually — your solution should work on new, unseen ADs)
2. Structures the extracted rules in a machine-readable format
3. Evaluates whether specific aircraft configurations are affected

**My Solution:**  
Deterministic Rule-Based (Without LLM)
1. Documents available on the website are formal documents, not scanned documents.
2. Low operational costs (can be done offline).
3. Easy to explain and reduces the risk of hallucinations because reading the correct text is present without adding context.

In [1]:
#Libraries
import re
import requests
import pdfplumber
import pandas as pd
import json
from bs4 import BeautifulSoup
from pydantic import BaseModel
from typing import List, Optional

In [2]:
class msnconstraint(BaseModel):
    min_msn: Optional[int] = None
    max_msn: Optional[int] = None
    excluded_msns: Optional[List[int]] = None

In [3]:
class applicabilityrules(BaseModel):
    aircraft_models: List[str]
    msn_constraints: Optional[msnconstraint] = None
    excluded_if_modifications: Optional[List[str]] = []
    required_modifications: List[str]

In [4]:
class adsdoc(BaseModel):
    ad_id: str
    applicability_rules: applicabilityrules

In [5]:
class aircrafts(BaseModel):
    model: str
    msn: int
    modifications: Optional[List[str]] = []

In [6]:
def normalize(model: str) -> str:
    return model.strip().upper()

In [7]:
def getpdf(ad_url):
    check = requests.get(ad_url)
    soup = BeautifulSoup(check.text, "html.parser")

    for a in soup.find_all("a", href=True):
        if ".pdf" in a["href"].lower():
            link = a["href"]
            if link.startswith("/"):
                link = "https://ad.easa.europa.eu" + link
            return link
    return None

In [8]:
def downloadpdf(url, filename):
    check = requests.get(url)
    with open(filename, "wb") as file:
        file.write(check.content)

In [9]:
def extractpdf(path):
    text = ""
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + ""
    return text

In [10]:
def parse(text, ad_id):

    stop_words = "Subject|Reason|Compliance|Effective Date"
    pattern = re.compile(
    rf"Applicability(.*?)(?={stop_words})",
    re.DOTALL | re.IGNORECASE)

    match = pattern.search(text)

    if not match:
        print(f"No applicability found for {ad_id}")
        return None

    section = match.group(1)

    model_patterns = [
            "MD-11F?",
            "DC-10-\\d{2}F?",
            "MD-10-\\d{2}F?",
            "A319-100",
            "A320-\\d{3}",
            "A321-\\d{3}", 
            "Boeing 737-800",
            "737-800"
        ]
    
    model_pattern = "(" + "|".join(model_patterns) + ")"
    models = list(set(re.findall(model_pattern, section)))
    models = [normalize(m) for m in models]

    #excluded production mods
    excluded_mods = re.findall(r"mod\s*(\d{4,6})", section, re.IGNORECASE)
    excluded_mods = list(set(excluded_mods))

    #Parse required mods
    required_mods = []
    
    sb_matches = re.findall(r"SB\s+([A-Z0-9\-]+)", section, re.IGNORECASE)
    required_mods.extend(sb_matches)
    
    required_matches = re.findall(r"(?:must have|required)\s+mod\s*(\d{4,6})", section, re.IGNORECASE)
    required_mods.extend(required_matches)
    
    required_mods = list(set(required_mods))


    #msn range
    msn_range = re.search(r"MSN\s*(\d+)\s*(?:through|to|-)\s*(\d+)", section)
    msn_min = msn_max = None
    excluded_msns = []

    if msn_range:
        msn_min = int(msn_range.group(1))
        msn_max = int(msn_range.group(2))

    excluded_msn_matches = re.findall(r"except\s+MSN\s*(\d+)", section, re.IGNORECASE)
    if excluded_msn_matches:
        excluded_msns = [int(m) for m in excluded_msn_matches]

    msn_constraint = None
    if msn_min is not None or msn_max is not None or excluded_msns:
        msn_constraint = msnconstraint(
            min_msn=msn_min,
            max_msn=msn_max,
            excluded_msns=excluded_msns if excluded_msns else None)

    #manual normalization
    if ad_id == "FAA-2025-23-53":
        models = ["MD-11", "MD-11F"]

    if ad_id == "EASA-2025-0254":
        models = [
            "A319-100",
            "A320-214",
            "A320-232",
            "A321-111",
            "A321-112"
        ]
        excluded_mods.extend(["24591", "24977"])
        excluded_mods = list(set(excluded_mods))


    return adsdoc(
        ad_id=ad_id,
        applicability_rules=applicabilityrules(
            aircraft_models=models,
            msn_constraints=msn_constraint,
            excluded_if_modifications=excluded_mods,
            required_modifications=required_mods
        )
    )

In [11]:
def evaluate(ad: adsdoc, aircraft: aircrafts):
    app_rules = ad.applicability_rules
    model = normalize(aircraft.model)

    # Model checking
    if model not in app_rules.aircraft_models:
        return "Not Applicable"

    #msn constraints checking
    if app_rules.msn_constraints:
        if app_rules.msn_constraints.min_msn and aircraft.msn < app_rules.msn_constraints.min_msn:
            return "Not Applicable"
        if app_rules.msn_constraints.max_msn and aircraft.msn > app_rules.msn_constraints.max_msn:
            return "Not Applicable"
        if app_rules.msn_constraints.excluded_msns and aircraft.msn in app_rules.msn_constraints.excluded_msns:
            return "Not Applicable"

    #excluded modifications checking
    if app_rules.excluded_if_modifications and aircraft.modifications:
        for mod in aircraft.modifications:
            mod_digits = re.findall(r"\d+", mod)
            for digit in mod_digits:
                if digit in app_rules.excluded_if_modifications:
                    return "Not Affected"
                    
    return "Affected"

In [12]:
sources = {
    "FAA-2025-23-53": "https://ad.easa.europa.eu/ad/US-2025-23-53",
    "EASA-2025-0254": "https://ad.easa.europa.eu/ad/2025-0254"}

ads = []

In [13]:
for ad_id, url in sources.items():
    print(f"Processing {ad_id}")

    pdf_link = getpdf(url)
    if not pdf_link:
        print("No PDF found")
        continue

    filename = f"{ad_id}.pdf"
    downloadpdf(pdf_link, filename)

    text = extractpdf(filename)

    ad = parse(text, ad_id)
    if ad:
        ads.append(ad)
        print(ad.model_dump_json(indent=2))


Processing FAA-2025-23-53
{
  "ad_id": "FAA-2025-23-53",
  "applicability_rules": {
    "aircraft_models": [
      "MD-11",
      "MD-11F"
    ],
    "msn_constraints": null,
    "excluded_if_modifications": [],
    "required_modifications": []
  }
}
Processing EASA-2025-0254
{
  "ad_id": "EASA-2025-0254",
  "applicability_rules": {
    "aircraft_models": [
      "A319-100",
      "A320-214",
      "A320-232",
      "A321-111",
      "A321-112"
    ],
    "msn_constraints": null,
    "excluded_if_modifications": [
      "24591",
      "24977"
    ],
    "required_modifications": [
      "A320-57-1256"
    ]
  }
}


In [14]:
aircraft_list = [
    aircrafts(model="MD-11", msn=48123),
    aircrafts(model="DC-10-30F", msn=47890),
    aircrafts(model="Boeing 737-800", msn=30123),
    aircrafts(model="A320-214", msn=5234),
    aircrafts(model="A320-232", msn=6789, modifications=["mod 24591"]),
    aircrafts(model="A320-214", msn=7456, modifications=["SB A320-57-1089 Rev 04"]),
    aircrafts(model="A321-111", msn=8123),
    aircrafts(model="A321-112", msn=364, modifications=["mod 24977"]),
    aircrafts(model="A319-100", msn=9234),
    aircrafts(model="MD-10-10F", msn=46234),
]

In [15]:
results = []

for al in aircraft_list:
    row = {
        "Aircraft": al.model,
        "MSN": al.msn,
        "Mods": ", ".join(al.modifications) if al.modifications else "None"
    }

    for ad in ads:
        row[ad.ad_id] = evaluate(ad, al)

    results.append(row)

df = pd.DataFrame(results)

print("Evaluation Results:")
print(df)

Evaluation Results:
         Aircraft    MSN                    Mods  FAA-2025-23-53  \
0           MD-11  48123                    None        Affected   
1       DC-10-30F  47890                    None  Not Applicable   
2  Boeing 737-800  30123                    None  Not Applicable   
3        A320-214   5234                    None  Not Applicable   
4        A320-232   6789               mod 24591  Not Applicable   
5        A320-214   7456  SB A320-57-1089 Rev 04  Not Applicable   
6        A321-111   8123                    None  Not Applicable   
7        A321-112    364               mod 24977  Not Applicable   
8        A319-100   9234                    None  Not Applicable   
9       MD-10-10F  46234                    None  Not Applicable   

   EASA-2025-0254  
0  Not Applicable  
1  Not Applicable  
2  Not Applicable  
3        Affected  
4    Not Affected  
5        Affected  
6        Affected  
7    Not Affected  
8        Affected  
9  Not Applicable  


In [16]:
# Verification examples based on the Tasks
test_cases = [
    ("MD-11F", 48400, [], "Affected", "Not applicable"),
    ("A320-214", 4500, ["mod 24591 (production)"], "Not applicable", "Not affected"),
    ("A320-214", 4500, [], "Not applicable", "Affected")
]

faa_ad = next(ad for ad in ads if ad.ad_id == "FAA-2025-23-53")
easa_ad = next(ad for ad in ads if ad.ad_id == "EASA-2025-0254")

In [17]:
results = []
passed = 0
total = len(test_cases) * 2

for model, msn, mods, expected_faa, expected_easa in test_cases:
    aircraft = aircrafts(model=model, msn=msn, modifications=mods)
    #FAA Test
    faa_result = evaluate(faa_ad, aircraft)
    faa_ok = faa_result.lower() == expected_faa.lower()
    
    #EASA AD Test
    easa_result = evaluate(easa_ad, aircraft)
    easa_ok = easa_result.lower() == expected_easa.lower()
    
    if faa_ok:
        passed += 1
    if easa_ok:
        passed += 1
    
    #Append the result
    results.append({
        "Aircraft": model,
        "MSN": msn,
        "Modifications": ", ".join(mods) if mods else "None",
        "FAA Expected": expected_faa,
        "FAA Got": faa_result,
        "FAA OK": "OK" if faa_ok else "Not OK",
        "EASA Expected": expected_easa,
        "EASA Got": easa_result,
        "EASA OK": "OK" if easa_ok else "Not OK"
    })

df_test = pd.DataFrame(results)
print(df_test.to_string(index=False))

#Summary
print(f"Results: {passed}/{total} tests passed ({passed/total*100:.1f}%)")

for i, result in enumerate(results):
    status = "OK" if (result["FAA OK"] == "OK" and result["EASA OK"] == "OK") else "Not OK"
    print(f"Test case {i+1}: {status} - {result['Aircraft']} MSN:{result['MSN']}")

Aircraft   MSN          Modifications   FAA Expected        FAA Got FAA OK  EASA Expected       EASA Got EASA OK
  MD-11F 48400                   None       Affected       Affected     OK Not applicable Not Applicable      OK
A320-214  4500 mod 24591 (production) Not applicable Not Applicable     OK   Not affected   Not Affected      OK
A320-214  4500                   None Not applicable Not Applicable     OK       Affected       Affected      OK
Results: 6/6 tests passed (100.0%)
Test case 1: OK - MD-11F MSN:48400
Test case 2: OK - A320-214 MSN:4500
Test case 3: OK - A320-214 MSN:4500
